<a href="https://colab.research.google.com/github/medicadex/DMC/blob/main/GENERAL_JOB_FORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df= pd.read_excel('General Job Form.xlsx')
df.head()

In [ ]:
# @title GENERAL JOB FORM

import pandas as pd

def get_output_filename():
    return input("Kindly input name of completed cleaned job form: ")

def select_columns_to_drop(df):
    print("Enter the numbers of the columns you want to drop (separated by comma):")
    for i, column in enumerate(df.columns, start=1):
        print(f"{i}. {column}")
    selected_column_numbers = input("Selected columns: ")
    selected_column_numbers = [int(num.strip()) for num in selected_column_numbers.split(',') if num.strip()]

    # Drop selected columns
    columns_to_drop = [df.columns[num - 1] for num in selected_column_numbers if 0 < num <= len(df.columns)]
    return df.drop(columns=columns_to_drop, axis=1)

def confirm_action(prompt):
    while True:
        response = input(prompt).strip().lower()
        if response in ['yes', 'no']:
            return response
        else:
            print("Please enter 'yes' or 'no'.")
def select_DMO(df):
    print("Select an option for DMO:")
    print("1. Choose a specific DMO")
    print("2. Select all DMOs")
    choice = input("Enter the number of your choice: ")

    if choice == '1':
        print("List of DMOs in the Dataset:")
        DMOs = df['DMO'].unique()
        for i, DMO in enumerate(DMOs, start=1):
            print(f"{i}. {DMO}")
        selected_number = input("Enter the number of the DMO you want to include: ")
        selected_DMO = DMOs[int(selected_number) - 1]  # Select the DMO based on the user input
        return df[df['DMO'] == selected_DMO]
    elif choice == '2':
        return df
    else:
        print("Invalid choice. Selecting all DMOs.")
        return df


def select_UT(df):
    print("Select an option for UT:")
    print("1. Choose a specific UT")
    print("2. Select all UTs")
    choice = input("Enter the number of your choice: ")

    if choice == '1':
        print("List of UTs in the Dataset:")
        UTs = df['UT'].unique()
        for i, UT in enumerate(UTs, start=1):
            print(f"{i}. {UT}")
        print()
        selected_numbers = input("Enter the numbers of the UTs you want to include (separated by comma), or 'all' for all UTs: ")
        if selected_numbers.lower() == 'all':
            return df
        else:
            selected_numbers = [int(num.strip()) for num in selected_numbers.split(',') if num.strip()]
            selected_UTs = [UTs[num - 1] for num in selected_numbers]
            return df[df['UT'].isin(selected_UTs)]
    elif choice == '2':
        return df
    else:
        print("Invalid choice. Selecting all UTs.")
        return df

def modify_DT_name(df):
    replace_strings = input("Enter the strings you want to replace with empty space in the DT_name column (comma-separated): ")
    if replace_strings:
        replace_list = [string.strip() for string in replace_strings.split(',') if string.strip()]
        for string in replace_list:
            df['DT_name'] = df['DT_name'].str.replace(string, '')
    return df

def select_feeder_names(df):
    print("Select feeder names from the list below (separate multiple names with comma):")
    feeder_names = df['Feeder_name'].unique()
    for i, feeder_name in enumerate(feeder_names, start=1):
        print(f"{i}. {feeder_name}")
    selected_names = input("\nEnter the numbers of the feeder names you want to select (separated by comma), or 'all' for all feeder names: ").strip()

    if selected_names.lower() == 'all':
        return df
    else:
        selected_indices = [int(num.strip()) - 1 for num in selected_names.split(',') if num.strip()]
        selected_feeder_names = [feeder_names[idx] for idx in selected_indices]
        return df[df['Feeder_name'].isin(selected_feeder_names)]

def main():
    output_file_name = ""  # Initialize output_file_name with an empty string

    # Load the Excel file into a Pandas DataFrame
    try:
        df = pd.read_excel("General Job Form.xlsx")
    except FileNotFoundError:
        print("Error: File not found. Make sure the file exists and the path is correct.")
        return

    # Prompt the user to select DMO
    df = select_DMO(df)
    input_column = "DMO"
    suggestions = df[input_column]


    # Prompt the user to select an option
    option = input("Select an option:\n1. DT_NAME\n2. FEEDER_NAME\n3. PAYMENT PLAN\n4. UT \nEnter the number of your choice: ")

    if option == '4':  # When option is 'UT'
        df = select_UT(df)
        input_column = 'UT'
        suggestions = df['UT'].unique()

    elif option == '1':  # When option is 'DT_NAME'
        df = select_UT(df)
        input_column = 'UT'
        suggestions = df['UT'].unique()
        input_column = "DT_name"
        input_letters = input(f"Enter at least four letters from the {input_column}: ")
        suggestions = df[df[input_column].str.contains(input_letters, case=False)][input_column].unique()
    elif option == '2':  # When option is 'FEEDER NAME'
        df = select_feeder_names(df)
        input_column = "Feeder_name"
        suggestions = df[input_column].unique()
    else:  # When option is 'PAYMENT PLAN' or invalid
        df = select_UT(df)
        input_column = 'UT'
        suggestions = df['UT'].unique()

    # Check if there are suggestions
    if len(suggestions) > 0:
        print(f"Suggestions for {input_column}:")
        for i, suggestion in enumerate(suggestions, start=1):
            print(f"{i}. {suggestion}")

        # Prompt the user to select suggestions
        selected_numbers = input(f"Enter the numbers of the suggestions you want to keep (separated by comma): ").split(',')
        try:
            selected_numbers = [int(num.strip()) for num in selected_numbers if num.strip()]
        except ValueError:
            print("Error: Please enter valid numbers separated by comma.")
            return

        # Validate selected numbers
        valid_numbers = [i for i in range(1, len(suggestions) + 1)]  # Adjust the range for valid indices
        while not all(num in valid_numbers for num in selected_numbers):
            print("Invalid selection. Please enter valid numbers from the list.")
            selected_numbers = input(f"Enter the numbers of the suggestions you want to keep (separated by comma): ").split(',')
            selected_numbers = [int(num.strip()) for num in selected_numbers if num.strip()]

        # Map selected indices back to corresponding UTs
        selected_UTs = [suggestions[num - 1] for num in selected_numbers]

        # Filter rows based on the selected values
        if option == '4':  # When option is 'UT'
            df = df[df['UT'].isin(selected_UTs)]
        else:  # When option is 'DT_NAME' or 'FEEDER_NAME'
            df = df[df[input_column].isin(selected_UTs)]
    else:
        print("No suggestions found.")

    # Perform general cleaning if option is '3' or invalid
    if option == '3':
        df = select_UT(df)

    # Confirm dropping rows with zero Closing_balance
    drop_zero_balance = confirm_action("Do you want to drop rows where Closing_balance is zero? (yes/no): ")
    if drop_zero_balance == 'yes':
        df = df[df['Closing_balance'] != 0]

    # Confirm dropping rows with negative Outstanding_debt
    drop_negative_debt = confirm_action("Do you want to drop rows where Outstanding_debt is less than or equal to zero? (yes/no): ")
    if drop_negative_debt == 'yes':
        df = df[df['Outstanding_debt'] > 0]

    # Sort the DataFrame alphabetically by DT_name and Customer_name
    df = df.sort_values(by=['DT_name', 'Customer_name'])

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # Modify DT_name
    # Print feeder names
    print("List of Feeder Names in the Dataset:")
    feeder_names = df['Feeder_name'].unique()
    for i, feeder_name in enumerate(feeder_names, start=1):
        print(f"{i}. {feeder_name}")
    print()
    # Modify DT_name
    df = modify_DT_name(df)

    # Drop columns
    df = select_columns_to_drop(df)

    # Insert code to modify the content of 'S/N' column with serial numbering
    df['S/N'] = range(1, len(df) + 1)

    # Determine the cleaning type for the output file name
    cleaning_type = "DT" if option == '1' else "FEEDER" if option == '2' else "PAYMENT PLAN" if option == '3' else "UT"

    # Get the name of the cleaned file from the user
    output_file_name = get_output_filename()

    # Display the cleaned DataFrame
    print(df)

    # Save the cleaned DataFrame to a new Excel file with the user-inputted name
    output_file_name += ".xlsx"
    df.to_excel(output_file_name, index=False)
    print(f"The cleaned DataFrame has been saved to '{output_file_name}'.")

if __name__ == "__main__":
    main()